### Check WD (change if necessary) and file loading

In [94]:
# Show current directory
import os
curr_dir = os.getcwd()
print(curr_dir)

# Check if the current WD is the file location
if "/src/hfo/snn" not in os.getcwd():
    # Set working directory to this file location
    file_location = f"{os.getcwd()}/thesis-lava/src/hfo/snn"
    print("File Location: ", file_location)

    # Change the current working Directory
    os.chdir(file_location)

    # New Working Directory
    print("New Working Directory: ", os.getcwd())

/home/monkin/Desktop/feup/thesis/lava/src/hfo/snn


### Load the UP and DOWN spikes from the CSV Files

In [95]:
from utils.input import read_spike_events
from utils.io import preview_np_array

# Call the function to read the spike events
up_spikes_file_path = "./data/fr_up_spike_train_3.csv"
up_spike_train = read_spike_events(up_spikes_file_path)

down_spikes_file_path = "./data/fr_down_spike_train_-3.csv"
down_spike_train = read_spike_events(down_spikes_file_path)

preview_np_array(up_spike_train, "Spike Events")
preview_np_array(down_spike_train, "Spike Events")

Spike Events Shape: (198, 2).
Preview: [[1.00537109e+03 6.30000000e+01]
 [1.00878906e+03 6.30000000e+01]
 [1.01220703e+03 6.30000000e+01]
 [1.01269531e+03 6.30000000e+01]
 [1.01611328e+03 6.30000000e+01]
 ...
 [1.14575684e+05 6.30000000e+01]
 [1.14576172e+05 6.30000000e+01]
 [1.14579102e+05 6.30000000e+01]
 [1.14582520e+05 6.30000000e+01]
 [1.14585938e+05 6.30000000e+01]]
Spike Events Shape: (196, 2).
Preview: [[1.00439453e+03 6.30000000e+01]
 [1.00732422e+03 6.30000000e+01]
 [1.01123047e+03 6.30000000e+01]
 [1.01416016e+03 6.30000000e+01]
 [1.01806641e+03 6.30000000e+01]
 ...
 [1.14574219e+05 6.30000000e+01]
 [1.14574707e+05 6.30000000e+01]
 [1.14577637e+05 6.30000000e+01]
 [1.14581055e+05 6.30000000e+01]
 [1.14584961e+05 6.30000000e+01]]


## Format the input data to a tensor that slayer can convert to Event

In [96]:
import numpy as np
# Create a numpy array of shape (1, height, time) ?
sampling_rate = 2048    # 2048 Hz
input_duration = 120 * (10**3)    # 120000 ms or 120 seconds

x_step = 1/sampling_rate * (10**3)  # 0.48828125 ms

# Create a list of time steps
x = [val for val in np.arange(x_step, input_duration + x_step, x_step)] 

In [97]:
# Set Time Window for visualization
start_time = 1000           # in ms
end_time = 1024         # in ms

In [98]:
from utils.io import preview_np_array

formatted_input = []

curr_up_spike_idx = 0
curr_down_spike_idx = 0

for i in range(len(x)):
    curr_time = x[i]
    if curr_time < start_time:
        continue
    if curr_time > end_time:
        break
    
    curr_trains = [0, 0]    # Up and Down Spike Trains
    
    # Check if the next up spike occured in this time step
    if up_spike_train[curr_up_spike_idx][0] <= curr_time:
        curr_trains[0] = 1  # Set Up Spike Train to 1
        curr_up_spike_idx += 1
    
    # Check if the next down spike occured in this time step
    if down_spike_train[curr_down_spike_idx][0] <= curr_time:
        curr_trains[1] = 1  # Set Down Spike Train to 1
        curr_down_spike_idx += 1
        
    formatted_input.append(curr_trains)

# convert to numpy array
formatted_input = np.array(formatted_input)

# preview_np_array(formatted_input, "Formatted Input")

# Transpose?
formatted_input = np.transpose(formatted_input)
preview_np_array(formatted_input, "Formatted Input")

Formatted Input Shape: (2, 50).
Preview: [[0 0 0 0 0 ... 0 0 0 0 0]
 [0 0 0 0 0 ... 0 0 0 0 0]]


In [99]:
from torch import tensor

# Convert to the shape accepted by slayer
spike_tensor =  tensor(formatted_input).cpu().data.numpy().reshape(1, formatted_input.shape[0], -1)
preview_np_array(spike_tensor, "Formatted Input")

Formatted Input Shape: (1, 2, 50).
Preview: [[[0 0 0 0 0 ... 0 0 0 0 0]
  [0 0 0 0 0 ... 0 0 0 0 0]]]


In [100]:
import torch

device = torch.device('cpu')
# device = torch.device('cuda') 

In [103]:
# import slayer from lava-dl
import lava.lib.dl.slayer as slayer
import matplotlib.pyplot as plt
from matplotlib import animation

# Create the event from the tensor
slayer_event = slayer.io.tensor_to_event(spike_tensor)

anim = slayer_event.anim(plt.figure(figsize=(10, 10)), frame_rate=100, pre_compute_frames=False)
anim.save(f'visualization/gifs/up_dn_1000_1024.gif', animation.PillowWriter(fps=2), dpi=300)

KeyboardInterrupt: 